In [11]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('ko_index_puls_stock.csv')


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_5944\1694634590.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ko_index_puls_stock.csv')


In [5]:
df.head()

,code,date,open,high,low,close,volume,change,open_ks11,high_ks11,low_ks11,close_ks11,volume_ks11,change_ks11,updown,comp,amount,marcap,market
0,5930,2001-06-11,4330.0,4380.0,4280.0,4280.0,493626.0,-0.042506,612.46,615.41,607.20,608.23,340123834.0,-0.0218,2.0,-13.55,1.711355e+12,2.137698e+14,KS11
1,5930,2001-06-11,4330.0,4380.0,4280.0,4280.0,493626.0,-0.042506,75.95,76.33,75.17,75.27,197160784.0,-0.0249,2.0,-1.92,1.274272e+12,1.879641e+14,KS200
2,5930,2001-06-12,4220.0,4300.0,4210.0,4220.0,483939.0,-0.014019,605.36,611.88,603.91,607.15,319877548.0,-0.0018,2.0,-1.08,1.579309e+12,2.134079e+14,KS11
3,5930,2001-06-12,4220.0,4300.0,4210.0,4220.0,483939.0,-0.014019,74.89,75.78,74.69,75.08,174177478.0,-0.0025,2.0,-0.19,1.063817e+12,1.875352e+14,KS200
4,5930,2001-06-13,4300.0,4310.0,4200.0,4280.0,472921.0,0.014218,611.79,614.05,607.48,614.05,392415108.0,0.0114,1.0,6.90,1.882786e+12,2.157223e+14,KS11


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8923264 entries, 0 to 8923263
Data columns (total 19 columns):
 #   Column       Dtype  
---  ------       -----  
 0   code         object 
 1   date         object 
 2   open         float64
 3   high         float64
 4   low          float64
 5   close        float64
 6   volume       float64
 7   change       float64
 8   open_ks11    float64
 9   high_ks11    float64
 10  low_ks11     float64
 11  close_ks11   float64
 12  volume_ks11  float64
 13  change_ks11  float64
 14  updown       float64
 15  comp         float64
 16  amount       float64
 17  marcap       float64
 18  market       object 
dtypes: float64(16), object(3)
memory usage: 1.3+ GB


In [8]:
# rsi 구하기
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()

    rs = avg_gain / (avg_loss + 1e-6)  # 0으로 나누는 오류 방지
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [9]:
# 공포지수 구하기
def compute_fear_index(df):
    df = df.copy()
    
    # 1. 변동성 (standard deviation of close over 10 days)
    df["volatility_10"] = df["close"].rolling(10).std()

    # 2. 거래량 변화율
    df["volume_change"] = df["volume"].pct_change()

    # 3. 음봉 비율 (최근 10일 중 종가 < 시가 비율)
    df["red_candle"] = (df["close"] < df["open"]).astype(int)
    df["red_ratio_10"] = df["red_candle"].rolling(10).mean()

    # 4. 모멘텀 (최근 10일 전 대비 가격 변화)
    df["momentum_10"] = df["close"] - df["close"].shift(10)

    # 5. 공포지수 계산 (모든 요소 정규화 → 곱 or 가중 평균)
    # 아래는 단순한 곱 기반 예시
    df["fear_index"] = (
        df["volatility_10"].fillna(0) *
        df["volume_change"].fillna(0) *
        df["red_ratio_10"].fillna(0) *
        (-df["momentum_10"].fillna(0))  # 가격이 내려가야 공포
    )

    return df


In [12]:
# 종목 코드별 그룹화 후 파생변수 계산
features = []

for code, group in df.groupby("code"):
    group = group.sort_values("date").reset_index(drop=True)

    # 이동평균선 (MA)
    group["MA_5"] = group["close"].rolling(window=5).mean()
    group["MA_20"] = group["close"].rolling(window=20).mean()

    # 지수이동평균선 (EMA)
    group["EMA_5"] = group["close"].ewm(span=5, adjust=False).mean()
    group["EMA_20"] = group["close"].ewm(span=20, adjust=False).mean()

    # RSI
    group["RSI_14"] = compute_rsi(group["close"], period=14)

    # 공포지수 추가
    group = compute_fear_index(group)
    features.append(group)

# 결과 병합
df_features = pd.concat(features, ignore_index=True)

# 저장
df_features.to_csv("ko_stock_with_features.csv", index=False)